In [1]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'SNP'

import pandas as pd
pd.options.display.max_columns=None

# Add `src` to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path
from pathlib import Path
if 'src' not in Path.cwd().parts:
    src_path = str(Path(get_path('purelib')) / '_src.pth')
    with open(src_path, 'w') as f:
        f.write(str(Path.cwd() / 'src\n'))

# Start the Jupyter loop
from ib_insync import util
util.startLoop()

In [2]:
# Imports 
from utils import Vars, get_pickle

# Set the root
from from_root import from_root
ROOT = from_root()

_vars = Vars(MARKET)
PORT = port = _vars.PORT
OPT_COLS = _vars.OPT_COLS[0]
DATAPATH = ROOT / 'data' / MARKET.lower()


# Build the Base

In [3]:
# imports
import glob

from utils import delete_files, assemble_snp_underlyings, assemble_nse_underlyings, pickle_me, make_chains, pickle_with_age_check
from loguru import logger
import logging
import asyncio

In [4]:
# inputs
PAPER = False

In [5]:
# Set variables
MARKET = MARKET.upper()
_vars = Vars(MARKET)

if PAPER:
    PORT = _vars.PAPER
else:
    PORT = _vars.PORT

CALLSTDMULT = _vars.CALLSTDMULT
PUTSTDMULT = _vars.PUTSTDMULT

DTESTDEVLOW = _vars.DTESTDEVLOW
DTESTDEVHI = _vars.DTESTDEVHI
DECAYRATE = _vars.DECAYRATE

# Set paths for nse pickles
unds_path = ROOT / 'data' / MARKET / 'unds.pkl'
chains_path = ROOT / 'data' / MARKET / 'df_chains.pkl'
lots_path = ROOT / 'data' / MARKET / 'lots.pkl'

qualified_puts_path = ROOT / 'data' / MARKET / 'df_qualified_puts.pkl'
qualified_calls_path = ROOT / 'data' / MARKET / 'df_qualified_calls.pkl'

opt_prices_path = ROOT / 'data' / MARKET / 'df_opt_prices.pkl'
opt_margins_path = ROOT / 'data' / MARKET / 'df_opt_margins.pkl'

naked_targets_path = ROOT / 'data' / MARKET / 'df_naked_targets.pkl'

# Delete log files
log_folder_path = ROOT / 'log' / str(MARKET.lower()+"*.log")
file_pattern = glob.glob(str(log_folder_path))

delete_files(file_pattern)

# Set the logger with logpath
IBI_LOGPATH = ROOT / 'log' / f'{MARKET.lower()}_ib.log'
LOGURU_PATH = ROOT / 'log' / f'{MARKET.lower()}_app.log'

util.logToFile(IBI_LOGPATH, level=logging.ERROR)
logger.add(LOGURU_PATH, rotation='10 MB', compression='zip', mode='w')

# Assemble underlyings
if MARKET == 'SNP':
    unds = asyncio.run(assemble_snp_underlyings(PORT))
else:
    unds = asyncio.run(assemble_nse_underlyings(PORT))

# pickle underlyings
# pickle_with_age_check(unds, unds_path, 0) # No need to age-check, for fresh base build
pickle_me(unds, unds_path)

# Make chains for underlyings and limit the dtes
df_chains = asyncio.run(make_chains(port=PORT,
                                    MARKET=MARKET, 
                                    contracts=list(unds.values())))
df_chains = df_chains[df_chains.dte <= _vars.MAXDTE].reset_index(drop=True)
pickle_with_age_check(df_chains, chains_path, 0)

Getting SNP chains: 100%|██████████| 10/10 [04:18<00:00, 25.88s/it]
